In [10]:
import anvil.server

anvil.server.connect("7WL7UQ2JVJJGQMZESH32T5PF-G37I6CATNM2222EM")

In [1]:
lat = 32.661343
lon = 51.680374
#Very slow
from pysolar.solar import *
import pytz
import datetime
import numpy as np
import pandas as pd

#Input
date = datetime.datetime(2017,1,1,12,0,0,0, pytz.UTC)
date_end = datetime.datetime(2018,1,12,0,0,0,0, pytz.UTC)
time_delta = datetime.timedelta(seconds = 3600 )
#print(get_azimuth(lat, lot, date))

# Creating an empty list
times = []

def azimuth(date):
    return get_azimuth(lat, lon, date)

def elevation(date):
    return get_altitude(lat, lon, date)

azimuth = np.vectorize(azimuth)
elevation = np.vectorize(elevation)

steps = int((date_end-date) / time_delta)
for step in range(0, steps):
    date += time_delta
    times.append(date)
#coord.append([get_altitude(32.661343, 51.680374, date), get_azimuth(32.661343, 51.680374, date)])

times = np.asarray(times)
result_array = np.array([times, elevation(times), azimuth(times)]).transpose()
#print(get_azimuth(32.661343, 51.680374, df.iloc[0,0].to_datetime()))
df = pd.DataFrame(data=result_array, columns = ["Time", "Elevation", "Azimuth"])
df
#print(df.head())

,Time,Elevation,Azimuth
0,2017-01-01 13:00:00+00:00,6.31717,237.628
1,2017-01-01 14:00:00+00:00,-4.93388,245.822
2,2017-01-01 15:00:00+00:00,-16.7498,253.1
3,2017-01-01 16:00:00+00:00,-29.0185,259.985
4,2017-01-01 17:00:00+00:00,-41.5513,267.106
...,...,...,...
9007,2018-01-11 20:00:00+00:00,-75.7294,317.538
9008,2018-01-11 21:00:00+00:00,-78.2837,21.905
9009,2018-01-11 22:00:00+00:00,-69.4052,62.7755
9010,2018-01-11 23:00:00+00:00,-57.4419,79.1528


In [11]:
from scipy.optimize import minimize

@anvil.server.callable
def output(area):
    df["Elevation"] = df["Elevation"].astype(np.float64)
    df["Azimuth"] = df["Azimuth"].astype(np.float64)
    df.loc[df["Elevation"] >= 0, "x_pos"] = math.cos(math.radians(df["Elevation"]))*math.sin(math.radians(df["Azimuth"]))
    df.loc[df["Elevation"] >= 0, "y_pos"] = math.cos(math.radians(df["Elevation"]))*math.cos(math.radians(df["Azimuth"]))
    df.loc[df["Elevation"] >= 0, "z_pos"] = math.sin(math.radians(df["Elevation"]))
    df.loc[df["Elevation"] >= 0, "Irradiance"] = 1045 * math.cos(math.pi/2 - math.radians(df["Elevation"]))**1.15
    
    panel_orientation = [32, 120]
    res = minimize(power_extraction, panel_orientation, method='SLSQP')
    #the optimal panel orientation
    panel_orientation = res.x
    energy_sqm_yearly = -res.fun*3600 #because its on an hourly basis
    panel_density = 0.52 #sqm panel per sqm area, from excel sheet, we could optimize the row spacing here as well if we wanted to
    efficiency = 0.14 #panel efficiency
    energy_out_area_yearly = energy_sqm_yearly*panel_density*area*efficiency
    kWh = energy_out_area_yearly/1000/3600 #kWh
    #print("Energy Output per Year: {:,.2f} MWh".format(kWh/1000))
    infeed_tariff = 0.2 #USD/kwH
    #print("Expected Revenue per Year: {:,.0f} USD".format(kWh*infeed_tariff))
    revenue = kWh*infeed_tariff
    return kWh, revenue
    
    
def power_extraction(panel_orientation):
    elevation = panel_orientation[0]
    azimuth = panel_orientation[1]
    x = math.cos(math.radians(elevation))*math.sin(math.radians(azimuth))
    y = math.cos(math.radians(elevation))*math.cos(math.radians(azimuth))
    z = math.sin(math.radians(elevation))
    scalar_product = df["x_pos"]*x + df["y_pos"]*y + df["z_pos"]*z
    power_out = df["Irradiance"].copy()
    power_out[scalar_product > 0] = scalar_product*df["Irradiance"]
    power_out[scalar_product <= 0] = 0
    total_power = power_out.sum()
    return -total_power

In [4]:
#df.loc[df["Elevation"] >= 0, "Effective_Elevation"] = math.degrees(math.atan(math.tan(math.radians(df["Elevation"]))/math.cos(math.radians(180-df["Azimuth"]))))

In [12]:
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
from IPython.display import display

In [13]:
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl,WidgetControl
from ipywidgets import IntSlider, ColorPicker, jslink
m = Map(center=(lat, lon), basemap=basemaps.Esri.WorldImagery, bounce_at_zoom_limits = False, maxZoom = 10)
zoom_slider = IntSlider(description='Zoom level:', min=0, max=18, value=12)
jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
m.add_control(widget_control1)

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False

}

m.add_control(draw_control)
def handle_draw(self, action, geo_json):
    global coordinates
    coordinates = geo_json["geometry"]["coordinates"]


draw_control.on_draw(handle_draw)
display(m)

Map(bounce_at_zoom_limits=False, center=[32.661343, 51.680374], controls=(ZoomControl(options=['position', 'zo…

In [14]:
def reproject(latitude, longitude):
    """Returns the x & y coordinates in meters using a sinusoidal projection"""
    from math import pi, cos, radians
    earth_radius = 6371009 # in meters
    lat_dist = pi * earth_radius / 180.0

    y = [lat * lat_dist for lat in latitude]
    x = [long * lat_dist * cos(radians(lat)) 
                for lat, long in zip(latitude, longitude)]
    return x, y
    

x = []
y = []
for row in coordinates[0]:
    x.append(row[0])
    y.append(row[1])
    
    
[x,y] = reproject(x,y)
area = PolyArea(x,y) # in sqm
print("The area is: {:.0f} m²".format(area))

NameError: name 'coordinates' is not defined

In [15]:
@anvil.server.callable
def mapplot():
    m = Map(center=(lat, lon), basemap=basemaps.Esri.WorldImagery, bounce_at_zoom_limits = False, maxZoom = 10)
    display(m)

In [16]:
from geopy.geocoders import Nominatim
@anvil.server.callable
def get_coord(name):
    gc = Nominatim(user_agent="test")
    location = gc.geocode(name)
    return location.latitude, location.longitude
